In [2]:
# ===========================================
# Install Required Packages
# ===========================================
! pip install -U torch transformers datasets scikit-learn langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [ ]:
import zipfile
import os
import joblib
import torch
from transformers import AutoTokenizer
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

# --- 1. Ask user for paths ---
print("Enter full paths for model, scaler, and tokenizer directory.")
model_path = input("Path to model file (.pkl): ").strip()
numeric_scaler_path = input("Path to numeric_scaler.pkl: ").strip()
tokenizer_dir = input("Path to tokenizer directory: ").strip()

# --- 2. Validate paths ---
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at: {model_path}")
if not os.path.exists(numeric_scaler_path):
    raise FileNotFoundError(f"Scaler file not found at: {numeric_scaler_path}")
if not os.path.exists(tokenizer_dir):
    raise FileNotFoundError(f"Tokenizer directory not found at: {tokenizer_dir}")

# --- 3. Load tokenizer, scaler, and model ---
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)
numeric_scaler = joblib.load(numeric_scaler_path)
model = torch.load(model_path, map_location=torch.device("cpu"), weights_only=False)
model.eval()

# --- 4. Create LangChain LLM wrapper ---
class TrainedModelLLM(LLM):
    """LangChain LLM wrapper for a trained BERT+numeric features model."""

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """
        Args:
            prompt: The input text. Must include numeric values in this format:
                    'feedback: <text> | features: val1,val2,val3,...'
        Returns:
            Prediction as a string (class label or probability).
        """
        # Parse numeric features from prompt
        if "| features:" in prompt:
            text_part, num_part = prompt.split("| features:")
            text_input = text_part.replace("feedback:", "").strip()
            numeric_strs = [x.strip() for x in num_part.strip().split(",")]
            numeric_values = list(map(float, numeric_strs))
        else:
            raise ValueError("Prompt must include '| features: num1,num2,...'")

        # Tokenize text
        encoded = tokenizer(
            text_input,
            truncation=True,
            padding=True,
            return_tensors="pt"
        )

        # Scale numeric features
        numeric_scaled = torch.tensor(
            numeric_scaler.transform([numeric_values]),
            dtype=torch.float
        )

        # Run inference
        with torch.no_grad():
            output = model(
                input_ids=encoded["input_ids"],
                attention_mask=encoded["attention_mask"],
                numeric_tensor=numeric_scaled
            )

        # Determine predicted class
        if isinstance(output, torch.Tensor):
            pred_class = output.argmax(dim=1).item()
        else:
            logits = getattr(
                output,
                "logits",
                output[0] if isinstance(output, (list, tuple)) else output
            )
            pred_class = logits.argmax(dim=1).item()

        return str(pred_class)

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": "Custom BERT + numeric model"}

    @property
    def _llm_type(self) -> str:
        return "custom_bert_numeric_llm"


# --- 5. Test the LLM wrapper ---
llm = TrainedModelLLM()
print("Enter Prompt")
prompt = ""
prediction = llm(prompt)
print("Prediction:", prediction)


Enter full paths for model, scaler, and tokenizer directory.
